---

# Suis la lumière!
### GoPiGo 101 - Série d'exercices 4
##### Manipulation de la **DEL** arrière.

Pour cet exercice, vous devez avoir installé et connecté la diode électroluminescente (DEL ou _LED_ en anglais) sur l'un des ports `AD1` ou `AD1`. [Voir cette image](https://gopigo3.readthedocs.io/en/master/api-basic/structure.html#hardware-ports).

---

Les fonctions suivantes permettent la manipulation de la DEL externe :
 - `easygopigo3.init_led` retourne un objet `easysensors.Led`
 - `Led.light_max` allume la DEL à la puissance maximum
 - `Led.light_off` ferme la DEL
 - `Led.light_on(power)` allume la DEL à la puissance déterminée
     - `power` : un entier entre 0 et 100
 - `Led.is_on` retourne si la DEL est allumée
     - retourne un booléen
 - `Led.is_off` retourne si la DEL est éteinte
     - retourne un booléen
 
**Important** : Vous remarquerez que la performance lumineuse de la DEL est plutôt faible et très directionnelle.


### Démonstration

In [ ]:
import easygopigo3 as gpg
import time

robot = gpg.EasyGoPiGo3()
led_port = 'AD2'
rear_led = robot.init_led(led_port)


time_to_wait = 2.5 # en secondes
print(f'Tentative d''ouverture de la DEL arrière à pleine puissance. ', end='')
rear_led.light_max()
print(f'Succès : {rear_led.is_on()}')
time.sleep(time_to_wait)

print(f'Tentative de fermeture de la DEL arrière. ', end='')
rear_led.light_off()
print(f'Succès : {rear_led.is_off()}')
time.sleep(time_to_wait)

time_to_wait = 1.5 # en secondes
for i in range(100,-20,-20):
    print(f"Tentative d'ouverture de la DEL arrière à {i}% de sa puissance. ", end='')
    rear_led.light_on(i)
    print(f'Allumée : {rear_led.is_on()}')
    time.sleep(time_to_wait)


del rear_led
del led_port
del robot
del time
del gpg

### Préparation
Faites la mise en place du code commun pour cette série d'exercices

In [ ]:
# Mise en place du code commun
import easygopigo3 as gpg
import time

robot = gpg.EasyGoPiGo3()
led_port = 'AD2'
rear_led = robot.init_led(led_port)

### Exercice 4.1.
Faites clignoter la DEL à une fréquence de 0.25 seconde pendant 20 secondes.

In [ ]:
# Solution très simple
cycle_duration = 0.25 # en secondes
half_cycle_duration = cycle_duration / 2. # en secondes

total_duration = 10. # en secondes
total_cycle = round(total_duration / cycle_duration)

for _ in range(total_cycle):
    rear_led.light_max()
    time.sleep(half_cycle_duration)
    rear_led.light_off()
    time.sleep(half_cycle_duration)

---
### Exercice 4.2.
Faites un programme qui assigne l'intensité de la LED dans le temps selon ce premier patron.
<pre>
     Intensité
         ^
         |
   100 % |           *                 * * *
    80 % |         * * *               * * *
    60 % |       * * * * *             * * *
    40 % |     * * * * * * *           * * *
    20 % |   * * * * * * * * *   * * * * * * * * *
     0 % | * * * * * * * * * * * * * * * * * * * * *
         +-----------+---------+---------+---------+---> temps en secondes
                    1.0       2.0       3.0       4.0 
</pre>

Ensuite, 3 fois selon ce patron.
<pre>
     Intensité
         ^
         | 
   100 % |                             * * * * * *
    90 % |                       * * * * * * * * * * * *
    80 % |                   * * * * * * * * * * * * * * * *
    70 % |                 * * * * * * * * * * * * * * * * * *
    60 % |               * * * * * * * * * * * * * * * * * * * *
    50 % |               * * * * * * * * * * * * * * * * * * * *
    40 % |               * * * * * * * * * * * * * * * * * * * *
    30 % |             * * * * * * * * * * * * * * * * * * * * * *
    20 % |           * * * * * * * * * * * * * * * * * * * * * * * *
    10 % |       * * * * * * * * * * * * * * * * * * * * * * * * * * * *
     0 % | * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
         +-----------+---------+---------+---------+---------+---------+---------> temps en secondes
                    0.5       1.0       1.5       2.0       2.5       3.0
</pre>

Tentez de trouver une façon simple et flexible de pouvoir déterminer des patrons et le nombre de fois qu'ils sont appliqués en boucle.

In [ ]:
# Solution

# Création de patrons
pattern_1 = [0, 20, 40, 60, 80, 100, 80, 60, 40, 20, 0, 20, 20, 20, 100, 100, 100, 20, 20, 20, 0]
pattern_2 = [0, 0, 0, 10, 10, 20, 30, 60, 70, 80, 80, 90, 90, 90, 100, 100, 100]
pattern_2.extend(pattern_2[::-1])


def apply_pattern_to_led(pattern, n_times : int = 1, intra_pattern_delay : float = 0.1):
    if not isinstance(n_times, int) or n_times < 1:
        raise ValueError('n_times must be an integer greater than 0')
    if not isinstance(intra_pattern_delay, float) or intra_pattern_delay < 0.1:
        raise ValueError('intra_pattern_delay must be an floating point greater or equal than 0.1')
        
    for _ in range(n_times):
        for intensity in pattern:
             rear_led.light_on(intensity)
             time.sleep(intra_pattern_delay)


# apply_pattern_to_led(pattern_1, 1, 0.2)
apply_pattern_to_led(pattern_2, 3, 0.1)


### Exercice 4.3.
L'exercice précédent s'appuie sur la création d'un patron déterminé et fixe. Cette approche est flexible mais moins intéressante à utiliser si on veut un changement d'intensité tout en douceur.

Faites un programme qui fait osciller la DEL selon une fonction sinusoïdale.

Rappel : 
 - la fonction `sin` prend en entrée un réel non borné : $(-\infty, +\infty)$
 - la fonction `sin` retourne un réel borné : $[-1.0, +1.0]$
 - pour cet exercice, seule la forme sinuosidale importe et non son sens mathématique lié à la trigonométrie
 
Suggestions :
 - créez un compteur et initialisez le à 0
 - faites incrémenter le compteur par un certain interval (prendre `0.001` pour débuter)
 - déterminer l'intensité de la DEL selon la valeur de ce compteur passé dans la fonction `sin`
 - attention, il faut adapter la plage de sortie de la fonction `sin` [-1, 1] à celle de l'entrée de la fonction `light_on(...)` [0, 100]
 
 Notes : Vous observerez que la DEL _clignote_ de façon très continue et non de façon agrupte. L'effet est très intéressant et permet de faire une interface utilisateur plus agréable pour les situations le demandant.

In [ ]:
# Solution
import math
counter = 0
increment = 0.001

while True:
    intensity = int((math.sin(counter) + 1.) / 2. * 100.)
    rear_led.light_on(intensity)
    counter += increment

### Exercice 4.4
Faites un programme qui permet d'opérer la DEL avec la télécommande.

Au début du programme, l'intensité initiale est de 50%. Ensuite, la télécommande permet de faire ces opérations :
 - `left` : 10%
 - `right` : 100%
 - `up` : une valeur aléatoire entre 0% et 100%
 - `down` : une valeur aléatoire entre 30% et 70%
 - `1` à `9` : la valeur en dizaine de pourcentage (1 => 10%, 2 => 20%, 3 => 30%, ...)
 - `0` : 100%
 - `ok` : 50%
 - `*` ou `#` : quitter le programme

In [ ]:
# Solution
import random

remote_control_port = 'AD1'
remote_control = robot.init_remote(port=remote_control_port)

key_action = {
    '' : lambda : -1,
    'left' :  lambda : 10,
    'right' :  lambda : 100,
    'up' :  lambda : random.randint(0, 100),
    'down' :  lambda : random.randint(30, 70),
    '0' :  lambda : 100,
    'ok' :  lambda : 50,
    '*' :  lambda : None,
    '#' :  lambda : None
}
for i in range(1, 10):
    key_action[str(i)] = lambda i=i: i * 10 # Portez attention à l'argument passé de la fonction lambda!
                                            # Cet argument permet de ne pas lier l'intensité retournée au i capturé du for mais plutôt à une copie unique de la valeur  

keycodes = ['', 'up', 'left', 'ok', 'right', 'down', '1', '2', '3', '4', '5', '6', '7', '8', '9', '*', '0', '#']
key = ''

print('Début du contrôle de la DEL...')
rear_led.light_on(current_led_intensity)
while current_led_intensity is not None:
    key = keycodes[remote_control.read()]
    current_led_intensity = key_action[key]()
    if current_led_intensity is not None and current_led_intensity >= 0:
        rear_led.light_on(current_led_intensity)
print('... fin du contrôle de la DEL.')